In [3]:
#Расчёт Пластинчато-трубчатого ТО (корридорный шаг)
import matplotlib.pyplot as pt
import numpy as n
import math as m
import json, CoolProp.CoolProp as CP
from CoolProp.CoolProp import PropsSI as p
CP.set_config_string(CP.ALTERNATIVE_REFPROP_PATH, 'C:\\maximov\\REFPROP\\')
from scipy.optimize import root


#исходные данные:

T12 = 32
T21 = 15
G1 = 6233
G2 = 10936
T11 = 73
T22 = 40
Q = 1142000000


P11 = 8000000
P21 = 200000
P12 = P11
P22 = P21

#Геометрия  #https://docs.cntd.ru/document/1200006715
Dvnes=0.01
DeltaTube=0.001
Dvnut=Dvnes-2*DeltaTube
step=3*Dvnes   #!!!!!!!!!!!!
Hchan=2*Dvnes
Deltaplate = 0.0005
N = 1
lambdaw = 20
row=7900
etta=0.9


CO2 = 'REFPROP::CO2'
O2 = 'REFPROP::Oxygen' #!
H2O = 'REFPROP::WATER'
ro11 = p('D','T', T11+273.15, 'P', P11, CO2)
ro12 = p('D','T', T12+273.15, 'P', P12, CO2)
ro21 = p('D','T', T21+273.15, 'P', P21, H2O)
ro22 = p('D','T', T22+273.15, 'P', P22, H2O)
ro1 = (ro11+ro12)/2
ro2 = (ro21+ro22)/2
nu11 = p('V','T', T11+273.15, 'P', P11, CO2)/ro11
nu12 = p('V','T', T12+273.15, 'P', P12, CO2)/ro12
nu21 = p('V','T', T21+273.15, 'P', P21, H2O)/ro21
nu22 = p('V','T', T22+273.15, 'P', P22, H2O)/ro22
nu1 = (nu11+nu12)/2
nu2 = (nu21+nu22)/2
lambda11 = p('L','T', T11+273.15, 'P', P11, CO2)
lambda12 = p('L','T', T12+273.15, 'P', P12, CO2)
lambda21 = p('L','T', T21+273.15, 'P', P21, H2O)
lambda22 = p('L','T', T22+273.15, 'P', P22, H2O)
lambda1 = (lambda11+lambda12)/2
lambda2 = (lambda21+lambda22)/2
Pr11 = p('PRANDTL','T', T11+273.15, 'P', P11, CO2)
Pr12 = p('PRANDTL','T', T12+273.15, 'P', P12, CO2)
Pr21 = p('PRANDTL','T', T21+273.15, 'P', P21, H2O)
Pr22 = p('PRANDTL','T', T22+273.15, 'P', P22, H2O)
Pr1 = (Pr11+Pr12)/2
Pr2 = (Pr21+Pr22)/2

dTmax = T11 - T22
dTmin = T12 - T21
dT = 0.9*(dTmax*1.00001 - dTmin) / (n.log(dTmax*1.00001 / dTmin))
##############################################
ddP1 = n.zeros(100000)
ddP2 = n.zeros(100000)
ddP4 = n.zeros(100000)
ddP6 = n.zeros(100000)
ddP8 = n.zeros(100000)
ddP10= n.zeros(100000)
dP11 = n.zeros(100000)
dP12 = n.zeros(100000)
NWW = n.zeros(100000)
NHH = n.zeros(100000)
NLL = n.zeros(100000)
Vv = n.zeros(100000)
Nsumms= n.zeros(100000)
Ff = n.zeros(100000)
i=0
print('go')
for w1 in n.arange(0.01,10,0.0001):
    i=i+1
    w2 = 2.5
    Re2 = w2*Dvnut/nu2
    Nu2 = 0.021*Re2**0.8*Pr2**0.43
    def func(x):
        delta = 10**-5
        left = x
        right = 1/(2*n.log10((2.51/(Re2*(x)**(1/2)))+(delta/3.7)))**2
        return left-right
    sol = root(func, 0.01)
    f2=float(sol.x)
    alpha2 = Nu2*lambda2/Dvnut
    Re1 = w1*Dvnes/nu1
    
    
    
    #Nu1 = 0.033*(Re1**0.9)*(Pr1**0.42) ###5
    #f1 = 7.2*(Re1**-0.42)
    #Nu1 = 0.04*(Re1**0.9)*(Pr1**0.42) ###4
    #f1 = 69*(Re1**-0.61)
    Nu1 = 0.038*(Re1**0.92)*(Pr1**0.42) ###3
    f1 = 22*(Re1**-0.41)
    #Nu1 = 0.035*(Re1**0.93)*(Pr1**0.42) ###25
    #f1 = 19*(Re1**-0.36)
    #Nu1 = 0.032*(Re1**0.95)*(Pr1**0.42) ###22
    #f1 = 21*(Re1**-0.34)
    
    alpha1 = Nu1*lambda1/Dvnes
    KF = Q/dT
    
    def func(x):
        Fvnes=x
        a1 = 1/KF
        a2 = 1/(etta*alpha1*Fvnes)
        a3 = ((1/2)*(n.log(Dvnes/Dvnut))*(Dvnes/(lambdaw*(Fvnes/2.316))))
        a4 = 1/(alpha2*Fvnes*0.345)
        return a1-a2-a3-a4
    sol = root(func, 0.002)
    x=sol.x
    Fvnes=float(x)
    Nsumm = m.ceil(Fvnes/(29.815*Dvnes**2))
    S2 = G2/(ro2*w2)
    NH = round(Nsumm*(n.pi*Dvnut**2/4)/S2)#
    S1 = G1/(ro1*w1)
    NL = round(Nsumm*0.04*step/S1) ##2N_яч S_труб (h_кан+δ_пл )
    NW = round(Nsumm/(NL*NH))
    W=NW*step*N
    H=NH*((Hchan+Deltaplate)*2)
    L=NL*step/N
    dP1 = f1*L/Dvnes*ro1*w1**2/2
    dP2 = f2*H/Dvnut*ro2*w2**2/2
    
    if round(dP1/100000,1)==round(0.01*P11/100000,1):     #0.01 - процент от входного 
        #print('1%',dP1,dP2,NW, NH, NL)
        ddP1[i] = dP1
    if round(dP1/100000,1)==round(0.02*P11/100000,1):     #0.01 - процент от входного 
        #print('2%',dP1,dP2,NW, NH, NL)
        ddP2[i] = dP1
    if round(dP1/100000,1)==round(0.03*P11/100000,1):     #0.01 - процент от входного 
        #print('4%',dP1,dP2,NW, NH, NL)
        ddP4[i] = dP1
    if round(dP1/100000,1)==round(0.04*P11/100000,1):     #0.01 - процент от входного 
        #print('6%',dP1,dP2,NW, NH, NL)
        ddP6[i] = dP1
    if round(dP1/100000,1)==round(0.05*P11/100000,1):     #0.01 - процент от входного 
        #print('8%',dP1,dP2,NW, NH, NL)
        ddP8[i] = dP1
    if round(dP1/100000,1)==round(0.06*P11/100000,1):     #0.01 - процент от входного 
        #print('10%',dP1,dP2,NW, NH, NL)
        ddP10[i] = dP1
    
    dP11[i] = dP1
    dP12[i] = dP2
    NWW[i] = NW
    NHH[i] = NH
    NLL[i] = NL
    
    
    
    Nsumms[i]=Nsumm
    Vv[i]=L*W*H
    Ff[i]=Fvnes
    if dP1>0.15*P11:
        break;
imax=i
print('done')




dP1p = [10**5 for i in range(imax)]
dP2p = [10**5 for i in range(imax)]
dP4p = [10**5 for i in range(imax)]
dP6p = [10**5 for i in range(imax)]
dP8p = [10**5 for i in range(imax)]
dP10p = [10**5 for i in range(imax)]

for i in range(imax):
    dP1p[i] = abs(0.01*P11 - ddP1[i])
j = n.argmin(dP1p)
print('1%', dP11[j],dP12[j],NWW[j], NHH[j], NLL[j])

for i in range(imax):
    dP2p[i] = abs(0.02*P11 - ddP2[i])
j = n.argmin(dP2p)
print('2%', dP11[j],dP12[j],NWW[j], NHH[j], NLL[j])

for i in range(imax):
    dP4p[i] = abs(0.03*P11 - ddP4[i])
j = n.argmin(dP4p)
print('4%', dP11[j],dP12[j],NWW[j], NHH[j], NLL[j])

for i in range(imax):
    dP6p[i] = abs(0.04*P11 - ddP6[i])
j = n.argmin(dP6p)
print('6%', dP11[j],dP12[j],NWW[j], NHH[j], NLL[j])

for i in range(imax):
    dP8p[i] = abs(0.05*P11 - ddP8[i])
j = n.argmin(dP8p)
print('8%', dP11[j],dP12[j],NWW[j], NHH[j], NLL[j])

for i in range(imax):
    dP10p[i] = abs(0.06*P11 - ddP10[i])
j = n.argmin(dP10p)
print('10%', dP11[j],dP12[j],NWW[j], NHH[j], NLL[j])
print('done')
#x = [dP[i] for i in range(1,imax)] 
#y = [Vv[i] for i in range(1,imax)]
#pt.figure(figsize=(8, 5))
#pt.plot(x, y)
#pt.grid(True)
##pt.ylim([0.5*10**7,2*10**7])
#pt.xlim([0,10*10**5])
#pt.show()
print((29.815*Dvnes**2))

go
done
1% 79972.05068384664 39843.05876663102 129.0 99.0 678.0
2% 159929.0229191721 38233.2382104035 101.0 95.0 864.0
4% 240059.51258646607 37428.327932289736 88.0 93.0 999.0
6% 319989.0076400019 37025.87279323286 79.0 92.0 1107.0
8% 399953.88833729894 36623.41765417599 73.0 91.0 1200.0
10% 479810.40138573264 36623.41765417599 68.0 91.0 1282.0
done
0.0029815
